In [71]:
! pip install openpyxl datetime pandas

In [63]:
import os
from openpyxl import Workbook
from datetime import datetime
import pandas as pd

In [64]:
user_input = 'prism_n1_le5_d202.log' # Enter path to log here
assert os.path.exists(user_input), "I did not find the file at "+str(user_input)
f = open(user_input,'r')
print("Log File found, beginning analysis...")
f.close()

Log File found, beginning analysis...


In [65]:
ProposerBlockMined = 'ProposerBlockMined'
ValidProposerBlockDownloaded = 'ValidProposerBlockDownloaded'
ProposerBlockConfirmed = 'ProposerBlockConfirmed'
ProposerBlockRolledBack = 'ProposerBlockRolledBack'
epoch = datetime.utcfromtimestamp(0)

hashToNumber = {}
hashToCreationTimestamp = {}
hashToCreationEvent = {}
hashToConfirmationTimestamp = {}
hashToRollbackTimestamp = {}

In [66]:
def getTimestamp(timestr):
    dateTimeObj = datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S,%f')
    delta = dateTimeObj - epoch
    return delta.total_seconds()*1000

In [67]:
with open(user_input, 'r') as f:
    for line in f:
        if "BENCHMARKING:" not in line:
            continue
        benchmarkSubstrings = line[line.find("BENCHMARKING"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        event = benchmarkSubstrings[1]
        blockNumber = benchmarkSubstrings[3]
        blockHash = benchmarkSubstrings[-1][1:-1]
        hashToNumber[blockHash] = blockNumber
        if event == ProposerBlockMined or event == ValidProposerBlockDownloaded:
            hashToCreationEvent[blockHash] = event
            hashToCreationTimestamp[blockHash] = timestamp
        if event == ProposerBlockConfirmed:
            hashToConfirmationTimestamp[blockHash] = timestamp
        if event == ProposerBlockRolledBack:
            hashToRollbackTimestamp[blockHash] = timestamp

f.close()

In [68]:
df = pd.DataFrame(columns=['Block Hash','Block Number','Timestamp - Creation','Event','Timestamp - Confirmation','Timestamp - Rollback'])

for index,key in enumerate(hashToNumber.keys()):
    df.loc[index] = [key,
        int(hashToNumber.get(key,'')),
        hashToCreationTimestamp.get(key,''),
        hashToCreationEvent.get(key,''),
        hashToConfirmationTimestamp.get(key,''),
        hashToRollbackTimestamp.get(key,'')]

df.sort_values('Block Number')

,Block Hash,Block Number,Timestamp - Creation,Event,Timestamp - Confirmation,Timestamp - Rollback
0,0x9398c82a4801990554f2738829276544af9cd54ba632...,1,1.638971e+12,ProposerBlockMined,1638971191615.0,
1,0xabd4128d8fd7cb53333cbc354952d677fd4a83ed03ea...,2,1.638971e+12,ProposerBlockMined,1638971216061.0,
2,0x38d416bf624b68fe0509db3b1a64b6e766882737cc73...,3,1.638971e+12,ProposerBlockMined,1638971272154.0,
3,0x74a3dc110de371b17f8c14f66fc401b8d10cba1a910b...,4,1.638971e+12,ProposerBlockMined,1638971379634.0,
4,0xa60d7824b6e3af7e0d04c4d3ed2c8818dc69385e9fc7...,5,1.638971e+12,ProposerBlockMined,1638971402024.0,
...,...,...,...,...,...,...
108,0x218faac1ab4e7001e1ba23242a0b418878e3adba1ab3...,109,1.638975e+12,ProposerBlockMined,,
109,0x89ffef551b64cb0ad1c7b0e82e2ddfb7c16b830363ec...,110,1.638975e+12,ProposerBlockMined,,
110,0x409cde7fa2cfce31527855c8c27c22719775b9afcd7b...,111,1.638975e+12,ProposerBlockMined,,
111,0x7e0e572d04199ca8c38c88837602de4a46dfcbdfd7ea...,112,1.638975e+12,ProposerBlockMined,,


In [69]:
df['Latency'] = df[df['Timestamp - Confirmation']!='']['Timestamp - Confirmation']-df[df['Timestamp - Confirmation']!='']['Timestamp - Creation']
df['Mining Time'] = df['Timestamp - Creation'].diff()
df['Confirmation Depth']=df['Latency']/df['Mining Time']

In [70]:
print("Mean Latency =",df['Latency'].dropna().mean())
print("Median Latency =",df['Latency'].dropna().median())
print("Mean Mining Time =",df['Mining Time'].dropna().mean())
print("Median Mining Time =",df['Mining Time'].dropna().median())
print("Mean Confirmation Depth =",df['Confirmation Depth'].dropna().mean())
print("Median Confirmation Depth =",df['Confirmation Depth'].dropna().median())

Mean Latency = 519466.35294117645
Median Latency = 516722.0
Mean Mining Time = 39663.25892857143
Median Mining Time = 29477.5
Mean Confirmation Depth = 43.08887581311689
Median Confirmation Depth = 19.20429331959471
